In [19]:
from __future__ import print_function 
import numpy as np
import tensorflow as tf 

import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import SimpleRNN, GRU, LSTM, Bidirectional

import keras.backend as K

## Load the dataset

In [20]:
f = np.load('data_and_embedding.npz')

In [60]:
num_words = int(f['num_words'])
embedding_dim = int(f['embedding_dim'])
max_sequence_length = int(f['max_sequence_length'])

data = f['data']
labels = f['labels']

embedding_matrix = f['embedding_matrix']

In [66]:
validation_split = 0.2 
epochs = 60

In [23]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(validation_split * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

## Conv

### Build the model 

In [61]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [71]:
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
# preds = embedded_sequences
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
print(x.shape)
x = Flatten()(x)
print(x.shape)
x = Dense(128, activation='relu')(x)
print(x.shape)
preds = Dense(6, activation='softmax')(x)

model_conv = Model(sequence_input, preds)

(?, 1, 128)
(?, ?)
(?, 128)


In [65]:
model_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 1000)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 35, 128)           82048     
__________

### Train the model

In [67]:
model_conv.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [68]:
import time 
start_time = time.time()

model_conv.fit(x_train, y_train,
          batch_size=128,
          epochs=epochs,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

Train on 311902 samples, validate on 77975 samples
Epoch 1/60
311902/311902 [==============================] - 1839s - loss: 0.6110 - acc: 0.7800 - val_loss: 0.3825 - val_acc: 0.8745
Epoch 2/60
311902/311902 [==============================] - 1727s - loss: 0.3258 - acc: 0.8943 - val_loss: 0.3335 - val_acc: 0.8945
Epoch 3/60
311902/311902 [==============================] - 1729s - loss: 0.2513 - acc: 0.9197 - val_loss: 0.2702 - val_acc: 0.9167
Epoch 4/60
311902/311902 [==============================] - 1718s - loss: 0.2100 - acc: 0.9330 - val_loss: 0.2483 - val_acc: 0.9238
Epoch 5/60
311902/311902 [==============================] - 1698s - loss: 0.1828 - acc: 0.9415 - val_loss: 0.2502 - val_acc: 0.9252
Epoch 6/60
311902/311902 [==============================] - 1675s - loss: 0.1630 - acc: 0.9478 - val_loss: 0.2497 - val_acc: 0.9283
Epoch 7/60
311902/311902 [==============================] - 1674s - loss: 0.1465 - acc: 0.9529 - val_loss: 0.2416 - val_acc: 0.9295
Epoch 8/60
311902/311902 

KeyboardInterrupt: 

### Save the model

In [ ]:
model_conv.save('models/Conv.h5')